In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import linear_reg_demo_grad_desc
import time

## Linear Regression Redux and Cost

We covered linear regression quite a bit in stats, it is one of the most simple and intuitive methods to make a predictive model - one that most of us did intuitively when doing a best fit line back in math class. 

The linear regression process we looked at used the algorithm of Linear Least Squares, which tries to create a model (the line) that has the lowest average squared distance from all the points. That is, the MSE between the model and the real values is a measurement of "badness" of the model - the smaller this value is, the better the model. We can also think of this MSE calculation as something called a Cost Function - the higher the cost, the worse the model is. 

The fitting part of a linear regression model is an attempt to minimize this cost function. The algorithm looks for the parameters (not hyperparameters) that minimize the cost; in a linear regression those parameters are the coefficients and the y-intercept. 

The linear least sqaures can calculate this directly, so the process of "looking" for the minimum is only one step. If you think back to logistic regression we can see a more dramatic example, the algorithm needs to test, try, and repeat. A decision tree is another example, the gini/entropy is the cost, and the algorithm searches through all potential splits until it finds the one that is best. That type of iteritive process is really common. 

### Cost Functions

The idea of cost is something that we will use throughout machine learning, it is critical for setting a goal that the algorithm can aim for when training. The cost function is just some function that measures the amount of error in a model, the lower the cost, the better a model we have. Usually this cost function is something that is a regular error metric that we have seen before - something like mean squared error for regression problems and accuracy or, more likely, cross-entropy for classifications. The cost to use is often something that we can specify as a hyperparameter when we create our models, such as the choice between gini and entropy in a tree. 

The cost function does not inherently need to be a regular error calculation, it could be almost any calculation at all. In certain weird situations the "goodness" of a model might not be measured accurately by a normal error calculation. Suppose you made a model to predict things on The Price is Right (https://priceisright.fandom.com/wiki/One_Bid); you want the model to make predictions that are close to the real price (probably well measured by MSE), however if the model predicts a price that is over, that's a tragedy and needs to be penalized severely. Maybe you'd want a model that calculated something more odd - MSE if the prediction is less than the real value, and residual^4 is the prediction is higher. 

#### Linear Regression Cost Function

The cost function is just our old friend the Mean Squared Error:

![Linear Regression Cost](images/lin_reg_cost.png "Linear Regression Cost" )

In [ ]:
#Generate some random data
X1 = 2 * np.random.rand(1000, 1)
y1 = 4 + 3 * X1**3 + 3*np.random.randn(1000, 1)


Generate a simple linear regression:

In [ ]:
lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X1, y1)

preds1 = lin_reg.predict(X1)
d1 = pd.DataFrame(X1, columns={"X"})
d1["Y_pred"] = preds1
d1["Y_real"] = y1
sns.lineplot(data = d1, x="X", y="Y_pred", color="red")
sns.scatterplot(data=d1, x="X", y="Y_real")

#### Regression Results

If we plot some data and generate a regression above, we have a solution that minimizes our cost function. There is no other model that we can create that has a lower mean squared residual between the model's predictions and the real values. Calculating the optimal model like we do here is great, however there are a couple of issues with that:

<ul>
<li>The computation cost when we have lots of data can become very large - growing exponentially. This can really matter when data grows massive. 
<li>Models that are not linear regression often can't be directly calculated (such as logistic regression). This is very common and we'll use gradient descent for things like neural networks later on. 
</ul>

To deal with situations where we can't directly compute the optimal solution we need a different approach. Rather than determining the correct solution directly, we will make an attempt, evaluate the cost, make a slightly different attempt, and try to improve until we can't find a better cost score with subsequent attempts. This approach is called...

## Gradient Descent

Gradient descent is a common approach to hunt for optimal solutions through an iteritive process. The process we can follow is:

<ul>
<li>Make an initial attempt to create a model, calculate the cost. This starting point is often random. 
<li>Compute the gradient - the derivitive or slope of the curve at that point. This indicates which direction to move. 
<li>Adjust the previous attempt, calculate the cost, compare to previous. The adjustment amount is determined by a value called the learning rate. 
<li>Repeat the previous step until moving does not improve the cost.
</ul>

We can visualize this process by looking at a curve of the cost function, and thinking about its derivitive or the slope. The gradient tells us two things:
<ul>
<li>Have we reached the best solution? If so, the gradient will be 0 indicating that we are at the minimum point on the curve. (i.e. there is no slope = we are at the bottom)
<li>Which direction to go? We always want to go down the curve.
</ul>

![Gradient Descent](images/grad_desc.png "Gradient Descent" )

When we've "settled" at the bottom, that is the lowest amount of cost, so there are no moves to make to find a better model. 

#### Gradient Descent - From Scratch

We can illustrate how gradient descent works. The file linear_reg_demo_grad_desc.py has an implementation of a linear regression that does gradient descent. I have modified it to return the set of predictions for each step of the training process. So what is happening in the background is:
<ul>
<li>Generate a linear regression. 
<li>Calculate the gradients.
<li>Update the weights to move against the gradient (down the curve).
<ul>
<li>The size of the jump is defined by the learning rate. Big rate, big move!
</ul>
<li>Repeat until the trials are done. (In real implementations you'll also stop when improvement ends.)
</ul>

This example is what is called Batch Gradient Descent - at every step the entire process is recalculated. 

Play around with the learning rates and iterations and see how it progresses:

In [ ]:
   
#Play around with some options and see the results!
learn_rate = .05 #Learning rate - how large to move each update.
iterations = 100 #How many iterations to run. 
num_show = 10 #How ofen to chart the line, i.e. 1 = every prediction, 10 = every 10th, etc.. 

train_rmses = []
test_rmses =[]
iters = []

plt.figure(figsize=(20,10))
X_train, X_test, y_train, y_test = train_test_split(X1, y1)
linreg = linear_reg_demo_grad_desc.LinearRegressionDemo(learning_rate=learn_rate, n_iters=iterations)
preds, test_preds = linreg.fit(X_train, y_train.ravel(), X_test)
sns.scatterplot(data=d1, x="X", y="Y_real")
for i in range(len(preds)):
    d_tmp = pd.DataFrame(X_train, columns={"X"})
    d_tmp["Y_pred"] = preds[i]
    iters.append(i)
    train_err = mean_squared_error(y_train, preds[i])
    train_rmses.append(train_err)
    test_err = mean_squared_error(y_test, test_preds[i])
    test_rmses.append(test_err)
    label = str(i) + " - RMSE:" + str(round(train_err, 3))
    if (i%num_show) == 0:
        sns.lineplot(data = d_tmp, x="X", y="Y_pred", label=label)
d_iters = pd.DataFrame(iters, columns={"Iteration"})
d_iters["Train"] = train_rmses
d_iters["Test"] = test_rmses

### Key Code Bits

We can look at the .py file and examine some of the key bits (you can open it directly for details and full code)

<b>perform gradient descent for n iterations</b>

```python
for _ in range(self.n_iters):
    # get y_prediction
    y_pred = self._get_prediction(X)
    # compute gradients
    dw, db = self._get_gradients(X, y, y_pred)
    # update weights & bias with gradients
    self._update_params(dw, db)
```
Here the fit function is pretty simple, we make a prediction, take that prediction to compute the gradients (slopes), then make and update to the weights...

```python
def _update_params(self, dw, db):
    self.weights -= self.lr * dw
    self.bias -= self.lr * db
```
Updating the weights is just incrementing them by the amount of the learning rate. 

```python
def _get_gradients(self, X, y, y_pred):
    # get distance between y_pred and y_true
    error = y_pred - y
    # compute the gradients of weight & bias
    dw = (1 / self.n_samples) * np.dot(X.T, error)
    db = (1 / self.n_samples) * np.sum(error)
    return dw, db
```
Calculating the gradients is just a recalculation of the gradients at our new point on the cost curve. The math uses vector math (dot products) that we can ignore for now. 

So the overall process, repeated for N iterations is to create a prediction (make a linear regression model), calculate the gradients on the cost curve at that point, update our position based on the position by the amount of the learning rate, and repeat. Eventually we will reach a point where the gradients, the slope of the location on the cost curve, is 0, then we won't be updating anymore - think about why by looking at the _update_params function. 

#### Errors

We can visualize the errors by iteration, or epoch as it is often called. Here we can see how long it takes for us to narrow in on a solution. 

I've graphed both the test and train errors, here they tend to be extremely close and often flip-flop in terms of accuracy depending on random splits. This is not a constant pattern, it is due to the data. 

In [ ]:
plt.figure(figsize=(20,5))
sns.lineplot(data=d_iters, x="Iteration", y="Train", label="Train")
sns.lineplot(data=d_iters, x="Iteration", y="Test", label="Test")

Our algorithm will work its way towards the solution. 

### Errors and Gradients

For a simple linear regression with one feature like we have here, the challenge is pretty simple. Our cost will be convex (shaped like a bowl), and we will always be able to find some solution, even if it takes a long time. With more complex datasets this isn't always true, we don't have a simple 2D curve, we have something that is in X dimension - maybe 100 or more at times. We can end up with curves that resemble this:

![Complex Gradient Descent](images/comp_grad_desc.png "Complex Gradient Descent" )

Here we have things like a local minima - a point at which the cost is minimized, but only locally, not overall. We don't want the algorithm to get 'stuck' in one of these holes, because we'll find a low cost, but not the lowest. 

Dealing with issues like this is there the learning rate comes in. By ensuring the learning rate is large enough, we have our attempts 'jump around' a little. This results in progress towards the minimum cost that is a little slower, but it also gives the function the ability to 'jump out' of problems like local minima. There's a balance with the learning rate, not too high, not too low. 

## Other Types of Descents

Batch gradient descent, like we did in the demo, suffers from the drawback that there is a lot of math involved in calculating the gradients, this can be slow with large amounts of data. In practice, there are a couple of things that we can do to speed this up. 

In the sample each time we iterated we calculated all of those gradients. For simple 2 varible datasets that's no big deal, but when the data gets massive, that's pretty slow. There are other implementations of gradient descent that apply the same concept, but take some shortcuts to lessen the amount of calculations needed. We can visualize how each 'hunts' the solution - the batch descent progresses steadily, the stochastic jumps around semi-randomly, and the mini-batch is between the two:

![Gradient Descent Patterns](images/grad_desc_patterns.png "Gradient Descent Patterns" )

#### Stochastic Gradient Descent

Stochastic (randomized) Gradient Descent, or SGD, speeds things up by just randomly selecting an instance and using that for the gradient calculation. The speed up comes from that massive reductions in the number of gradients calculated - instead of doing it for each record, SGD does it only once. Much faster. 

The downside to this is that there is much more randomness, and progress towards the solution isn't linear since you might randomly choose a point that is out of the way. This introduces some noise to the process, but the savings in computation time makes it generally worth it. 

#### Mini Batch Gradient Descent

MBGD combines the ideas of batch and SGD - at each step a small random subset of the data is used for the gradient calculations. 

#### Gradients and Outcomes

Note that the idea of all these algoritms is to generate a final model that is nearly the same, only the path there is different. Gradient descent isn't attempting to find a better model than linear regression (in a regular linear regression we can calculate the best model in a closed for solution), it is attempting to use a different method to reach that goal model. This idea of "narrowing in" on a solution can become even more useful when we don't have a solution that can be directly calculated - as long as we can set the cost function that defines accuracy, we can work our way towards an optimal solution. 



![Gradient Descent Outcomes](images/grad_desc_usage.png "Gradient Descent Outcomes" )

### SGD in Practice

We do not need to implement this descent process by hand in practice, we can use the built in SKlearn modules, and the idea is also built into several other algorithms. For linear regression we can use SGDregressor, a generic implementation of SGD.


We can do some gradient descent with actual data. 

In [ ]:
df = pd.read_csv("data/bodyfat.csv")
df.describe().T

In [ ]:
#Create Datasets
y2 = np.array(df["BodyFat"]).reshape(-1,1)
X2 = np.array(df.drop(columns={"BodyFat"}))
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=.3)

### Linear Regression Control Sample

We can generate a standard linear regression model first, and see what the results are. 

In [ ]:
#Closed Form Linear Regression
pipe_LR_steps = [('scale', MinMaxScaler()), ('model', LinearRegression())]
pipe_LR = Pipeline(pipe_LR_steps)

start = time.process_time()
pipe_LR.fit(X_train2, y_train2)
print(time.process_time() - start)

print('Training CrossVal Score:', cross_val_score(pipe_LR, X_train2, y_train2, cv=5))
print('Testing score:', pipe_LR.score(X_test2, y_test2))


### SGD Regression

Now we can create a model using gradient descent. There are a few options that we can specify in the SGD that are important:
<ul>
<li>eta0 - the initial learning rate. 
<li>learning_rate - how the learning rate is managed. In the examples we made by hand, the learning rate was constant. The SGD implementation provides for a way to adapt the learning rate - as you get closer to a solution, the learning rate slows. The defult is "invscaling", defined as: eta = eta0 / pow(t, power_t). t = number of updates, power_t = the exponent for inverse scaling learning rate.
<ul>
<li>This has the effect of using large learning rates to quickly narrow down to a close solution, hopefully taking advantage of both that speed increase and the ability to "jump" out of local minima. As the algorithm progresses, the rate slows to close in directly on a solution. 
</ul>
<li>early_stopping - should the algorithm stop when it fails to improve. This will set aside a validation dataset, and if predictions for this set stop improving, end the training. If we look back to the demo, at some point the error flattens out and doesn't change much, early stopping stops at this point. We'll look at early stopping in more detail later on. 
<li>penalty - SGD applies regularization by default, which we'll discuss next time.
</ul>

<b>Note: Scaling values is very important in SGD algorithms. We'll probably get poor results if we forget.</b>

In [ ]:
#SGD Regressor. 
pipe_SGD_steps = [('scale', MinMaxScaler()), ('model', SGDRegressor(max_iter=10000, eta0=.1))]
pipe_SGD = Pipeline(pipe_SGD_steps)

start = time.process_time()
pipe_SGD.fit(X_train2, y_train2.ravel())
print(time.process_time() - start)

#Print best model and test score
print('Training CrossVal Score:', cross_val_score(pipe_SGD, X_train2, y_train2.ravel(), cv=5))
print('Testing score:', pipe_SGD.score(X_test2, y_test2))

Note in the two examples above I added a timer. Play around with the learning rate and see what impact it has on the speed of SGD as compared to regular linear regression. 

## Working Example

Predict car prices using an SGD Regressor. 

In [ ]:
df3 = pd.read_csv("data/CarPrice_Assignment.csv")
df3.head()

In [ ]:
#Describe/info data - look for data types as compared with dictionary


<b>Note: There is probably a large amount of multicollinearity here. We'll try this example again with regularization soon, which is one way to deal with it in practice</b>

Also, depending on randomness there may be a need to deal with rare categorical values. Keep this in mind, you should be able to find 2+ solutions by Googling the error. 

In [ ]:
# Start to preprocess and model. 
# Use columnn transformer as there are mixed feature types
